<a href="https://colab.research.google.com/github/wandb/edu/blob/main/mlops-001/lesson1/02_Split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{course-lesson1} -->

# Data preparation

<!--- @wandbcode{course-lesson1} -->

In this notebook we will prepare the data to later train our deep learning model. To do so,

- we will start a new W&B `run` and use our raw data artifact
- split the data and save the splits into a new W&B Artifact
- join information about the split with our EDA Table

In [ ]:
import os, warnings
import wandb

import pandas as pd
from fastai.vision.all import *
from sklearn.model_selection import StratifiedGroupKFold

import params
warnings.filterwarnings('ignore')

In [ ]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="data_split")

Let's use artifact we previously saved to W&B (we're storing artifact names and other global parameters in `params`).

In [ ]:
raw_data_at = run.use_artifact(f'{params.RAW_DATA_AT}:latest')
path = Path(raw_data_at.download())

In [ ]:
path.ls()

To split data between training, testing and validation, we need file names, groups (derived from the file name) and target (here we use our rare class bicycle for stratification). We previously saved these columns to EDA table, so let's retrieve it from the table now. 

In [ ]:
fnames = os.listdir(path/'images')
groups = [s.split('-')[0] for s in fnames]

In [ ]:
orig_eda_table = raw_data_at.get("eda_table")

In [ ]:
y = orig_eda_table.get_column('bicycle')

Now we will split the data into train (80%), validation (10%) and test (10%) sets. As we do that, we need to be careful to:

- *avoid leakage*: for that reason we are grouping data according to video identifier (we want to make sure our model can generalize to new cars or video frames)
- handle the *label imbalance*: for that reason we stratify data with our target column

We will use sklearn's `StratifiedGroupKFold` to split the data into 10 folds and assign 1 fold for test, 1 for validation and the rest for training.

In [ ]:
df = pd.DataFrame()
df['File_Name'] = fnames
df['fold'] = -1

In [ ]:
cv = StratifiedGroupKFold(n_splits=10)
for i, (train_idxs, test_idxs) in enumerate(cv.split(fnames, y, groups)):
    df.loc[test_idxs, ['fold']] = i

In [ ]:
df['Stage'] = 'train'
df.loc[df.fold == 0, ['Stage']] = 'test'
df.loc[df.fold == 1, ['Stage']] = 'valid'
del df['fold']
df.Stage.value_counts()

In [ ]:
df.to_csv('data_split.csv', index=False)

We will now create a new artifact and add our data there. 

In [ ]:
processed_data_at = wandb.Artifact(params.PROCESSED_DATA_AT, type="split_data")

In [ ]:
processed_data_at.add_file('data_split.csv')
processed_data_at.add_dir(path)

Finally, the split information may be relevant for our analyses - rather than uploading images again, we will save the split information to a new table and join it with EDA table we created previously. 

In [ ]:
data_split_table = wandb.Table(dataframe=df[['File_Name', 'Stage']])

In [ ]:
join_table = wandb.JoinedTable(orig_eda_table, data_split_table, "File_Name")

Let's add it to our artifact, log it and finish our `run`. 

In [ ]:
processed_data_at.add(join_table, "eda_table_data_split")

In [ ]:
run.log_artifact(processed_data_at)
run.finish()